<a href="https://colab.research.google.com/github/lucas-azdias/Big-Data/blob/main/TDE3%20-%20Projeto%20Spark/TDE3%20-%20Projeto%20Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Processamento de Big Data com PySpark

In [1]:
!pip install PySpark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for PySpark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=b1c1d237b750fee559c62580c1b2705a869112294b032f53514eb52d69d10fae
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built PySpark


In [2]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Censo escolar 2021").master('local[*]').getOrCreate()
sc = spark.sparkContext

In [3]:
# RDD with columns separated and header filtered
rdd = sc.textFile("censo_escolar_2021.csv").map(lambda x: x.split(";")).filter(lambda x: x[0] != "NU_ANO_CENSO")

### 1. (1 ponto) Número de escolas em Curitiba;

In [4]:
rdd_cwb = rdd.filter(lambda x: x[4].upper() == "PR" and x[6].upper() == "CURITIBA")
amount_rdd__cwb = rdd_cwb.count()
print(amount_rdd__cwb)

1181


### 2. (1 ponto) Número de escolas por Região Geográfica (NO_REGIAO) ordenado alfabeticamente de acordo com o nome da região;

In [5]:
rdd_region = rdd.map(lambda x: (x[1], 1))
rdd_region = rdd_region.reduceByKey(lambda x, y: x + y)
rdd_region = rdd_region.sortByKey()
rdd_region.collect()

[('Centro-Oeste', 11659),
 ('Nordeste', 79039),
 ('Norte', 26095),
 ('Sudeste', 75329),
 ('Sul', 29018)]

### 3. (2 pontos) Nome (NO_ENTIDADE), município (NO_MUNICIPIO) e quantidade de docentes da escola que possui o maior número de professores somando as categorias de Educação Básica (QT_DOC_BAS), Ensino Fundamental (QT_DOC_FUND) e Ensino Médio (QT_DOC_MED);

In [6]:
empty_to_zero = lambda x: x if x != "" else 0

rdd_prof = rdd.map(lambda x: ((x[14], x[6]), (x[338], x[342], x[345])))
rdd_prof = rdd_prof.mapValues(lambda x: int(empty_to_zero(x[0])) + int(empty_to_zero(x[1])) + int(empty_to_zero(x[2])))
higher_rdd_prof = rdd_prof.reduce(lambda x, y: x if x[1] > y[1] else y)
print(higher_rdd_prof)

(('EDUCACAO INFANTIL MUNICIPAL BALAO MAGICO', 'Bom Jardim de Minas'), 623)


### 4. (2 pontos) Quantidade média de matrículas de Ensino Médio (QT_MAT_MED) em relação ao total de escolas por Região Geográfica (NO_REGIAO);

In [14]:
rdd_mat = rdd.filter(lambda x: x[305] != "")
rdd_mat = rdd_mat.map(lambda x: (x[1], (int(x[305]), 1)))
rdd_mat = rdd_mat.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
rdd_mat = rdd_mat.mapValues(lambda x: x[0] / x[1])
rdd_mat = rdd_mat.sortByKey()
rdd_mat.collect()

[('Centro-Oeste', 62.39879681738793),
 ('Nordeste', 37.01694021488366),
 ('Norte', 38.21229861583844),
 ('Sudeste', 50.977217603666595),
 ('Sul', 38.790391988107345)]

### 5. (2 pontos) Quantidade de escolas por Tipo da Localização (TP_LOCALIZACAO) e Tipo da Dependência Administrativa (TP_DEPENDENCIA);

In [8]:
rdd_loc = rdd.map(lambda x: ((int(x[17]), int(x[15])), 1))
rdd_loc = rdd_loc.reduceByKey(lambda x, y: x + y)
rdd_loc = rdd_loc.sortBy(lambda x: x[1], ascending=False)
rdd_loc.collect()

[((2, 3), 70091),
 ((1, 3), 64278),
 ((1, 4), 51559),
 ((1, 2), 26796),
 ((2, 2), 6768),
 ((2, 4), 934),
 ((1, 1), 619),
 ((2, 1), 95)]

### 6. (2 pontos) Quantidade de escolas por Sigla da Unidade da Federação (SG_UF) e Tipo de Dependência Administrativa (TP_DEPENDENCIA), ordenada de acordo com as maiores quantidades.

In [9]:
rdd_fed = rdd.map(lambda x: ((x[4], int(x[15])), 1))
rdd_fed = rdd_fed.reduceByKey(lambda x, y: x + y)
rdd_fed = rdd_fed.sortBy(lambda x: x[1], ascending=False)
rdd_fed.collect()

[(('BA', 3), 15659),
 (('SP', 3), 13822),
 (('SP', 4), 13550),
 (('MG', 3), 13363),
 (('MA', 3), 11432),
 (('PA', 3), 10371),
 (('CE', 3), 7255),
 (('SP', 2), 6572),
 (('RJ', 3), 6368),
 (('MG', 4), 6259),
 (('PE', 3), 6184),
 (('RS', 3), 5867),
 (('PR', 3), 5487),
 (('RJ', 4), 5301),
 (('PI', 3), 5281),
 (('AM', 3), 5081),
 (('MG', 2), 4319),
 (('SC', 3), 4290),
 (('PB', 3), 3895),
 (('RN', 3), 3435),
 (('BA', 4), 3297),
 (('PE', 4), 3216),
 (('RS', 4), 3165),
 (('ES', 3), 2752),
 (('GO', 3), 2751),
 (('CE', 4), 2604),
 (('RS', 2), 2549),
 (('AL', 3), 2474),
 (('PR', 4), 2399),
 (('PR', 2), 2164),
 (('MT', 3), 1823),
 (('SC', 4), 1608),
 (('SE', 3), 1572),
 (('RJ', 2), 1547),
 (('TO', 3), 1534),
 (('MA', 4), 1394),
 (('SC', 2), 1371),
 (('GO', 4), 1334),
 (('MA', 2), 1305),
 (('BA', 2), 1168),
 (('PE', 2), 1083),
 (('PA', 4), 1046),
 (('GO', 2), 1040),
 (('PB', 4), 1022),
 (('AC', 3), 1009),
 (('RN', 4), 1008),
 (('MS', 3), 993),
 (('PA', 2), 954),
 (('MT', 2), 824),
 (('RN', 2), 821)